In [93]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
!rm output/dynamic_delta_fits/??/*
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

In [ ]:
iOption0 = 1
is_sigmalised = 0
sim = 1
new = 0

Min_Entries = 100

In [ ]:
isigma = 2
N_files = 12
N_layers = 16
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if new: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NewHadron/rungroup/"
file_names="my-12th_QA_19423_1078runs_rg" #my-25th_QA_19197_1000runs
if not new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [96]:
if sim :
    file_path="../sim/output/Newembed/deltas/"
    file_names="piminus_DCA_50M_V1" #piminus_DCA_50M_V0_100mrad ccbar_50M_v0 piminus_DCA_50M_V0

In [97]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if new and not sim: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

../sim/output/Newembed/deltas/piminus_DCA_50M_V10.root


In [98]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.35, 0.65, 1.0, 1.4, 1.8, 2.2, 2.7, 3.4, 4.35]) 
pt_boarders =  array('d', [0.2, 0.5, 0.8, 1.2, 1.6, 2.0, 2.4, 3.0, 3.8, 4.9])


In [99]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]
arms = N_layers/8

In [100]:
def emc_fitter(InHist, iphi, seed=1, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*ilayer*100
    iilayer = int(ilayer/arms)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if new: central_bean_prev = (0.002*phi_bins[2*iphi]-0.001-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(ilayer)+"/"+str(iphi)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        if emc_var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{ilayer}_{iphi}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        g_gaus2.SetParameter(1,0.5*(boards_fit2[iilayer][0]+boards_fit2[iilayer][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[iilayer][1]-boards_fit2[iilayer][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/100,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [101]:
def emc_centr_fitter(ilayer=1,InHist=hists_emc_read, phibins=[0,1,2,3,4,5], sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for iFile in range(N_files):
        out_hists1 = []
        for iphi in phibins:
            out_hists1.append(emc_fitter(InHist[iFile][ilayer][iOption0+is_sigmalised*2], iphi, iFile, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [102]:
#Ntr = N_files
Ntr = N_layers
pool = mp.Pool(Ntr)
#output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
with pool:
    output_array = pool.map(emc_centr_fitter, [i for i in range(Ntr)])
pool.close()

In [103]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(N_files):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [104]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [105]:
g_sigma = []
for ifile in range(N_files):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    if N_files>1:
        c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",225*N_layers,1900)
        c3.Divide(int(N_layers/4),4)
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        g_sigma_layer = []
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

In [106]:
print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_ilayer in fit_functions_ifile:
        print("{")
        for fits in fit_functions_ilayer:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][16][6][3]={
{
{
{0.014434, 0.014444, -3.996833},
{0.192418, 0.023305, -2.450383},
{0.508524, 0.010426, -4.443584},
{1.066099, 0.000869, -5.929257},
{0.095171, 0.015685, -3.845764},
{0.300741, 0.017093, -3.677289},
},
{
{0.041409, 0.019568, -3.360316},
{0.018819, 0.020036, -3.294804},
{1.307306, 0.83644, -3.23978},
{0.144815, 0.000286, -6.247217},
{0.095729, 0.020521, -3.224938},
{0.293628, 0.123548, -3.299745},
},
{
{0.006135, 0.027103, 10.23},
{0.000258, 1.090149, -0.195344},
{0.189684, 0.877582, -0.239752},
{2e-06, 1.06227, -0.193218},
{0.412392, 0.711848, -0.430825},
{0.96411, 4.9e-05, 10.23},
},
{
{0.426111, 0.877863, -0.407312},
{0.288514, 0.934591, -0.285584},
{0.0, 1.177722, -0.186667},
{0.340163, 0.833843, -0.306131},
{0.494312, 0.737044, -0.458724},
{0.005955, 1.193199, -0.207221},
},
{
{0.001575, 6.052327, -10.089009},
{3e-06, 19.999954, -7.851444},
{0.099291, 0.300344, -4.751},
{2.8e-05, 17.441982, -9.290083},
{0.0, 0.000357, 10.23},
{0.122665, 2e-06, -2.0

In [107]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0<2:
                    fit_functions1[-1].SetParameter(2,-1)
                    fit_functions1[-1].SetParLimits(2,-1.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-20,20)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,-20,20)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-0.5,0.5)
                    
                    fit_functions1[-1].FixParameter(2,-1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [108]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    if N_files>1:
        c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",225*N_layers,1900)
        c3.Divide(int(N_layers/4),4)
    g_mean = [[[[]]*N_phi]*N_layers]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean[ifile][ilayer][iphi].append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            
            Format_Graph(g_mean[ifile][ilayer][iphi][-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            g_mean[ifile][ilayer][iphi][-1].Draw('P')
            if not is_sigmalised: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.7,1)
            else: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.4,2)
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][ilayer][iphi][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [109]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_layers}][{N_phi}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iilayer = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iilayer%2)].GetParameter(0),6)}, {round(fits[3-int(iilayer%2)].GetParameter(1),6)}, {round(fits[3-int(iilayer%2)].GetParameter(2),6)}"+"},")
            iilayer+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_ilayer in fit_functions_ifile:
            print("{")
            for fits in fit_functions_ilayer:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

const float phi_sMean_pt_params[1][16][3]={
{
{0.8, -1.05, -1.0},
{-0.8, 1.05, -1.0},
{0.420742, -0.533233, -1.0},
{-0.499998, 0.008258, -1.0},
{0.374169, -0.187563, -1.0},
{-0.8, 1.05, -1.0},
{-0.110212, 0.127368, -1.0},
{-0.499999, 0.456571, -1.0},
{0.8, -1.05, -1.0},
{-0.8, 1.05, -1.0},
{-0.207849, 0.386191, -1.0},
{-0.281116, 0.234073, -1.0},
{0.171805, -0.613946, -1.0},
{-0.350152, 6.342522, -1.0},
{-0.005173, 0.220997, -1.0},
{0.078112, -0.265733, -1.0},
},
};
sigma_pt_params[1][16][6][3]={
{
{
{0.8, -1.05, -1.0},
{0.8, -1.05, -1.0},
{0.8, -1.05, -1.0},
{0.8, -1.05, -1.0},
{0.8, -1.05, -1.0},
{0.8, -1.05, -1.0},
},
{
{-0.8, 1.05, -1.0},
{-0.8, 1.05, -1.0},
{-0.8, 1.05, -1.0},
{-0.8, 1.05, -1.0},
{-0.8, 1.05, -1.0},
{-0.8, 1.05, -1.0},
},
{
{-0.5, -1.909393, -1.0},
{-0.5, -1.202286, -1.0},
{-0.434835, -0.059941, -1.0},
{0.420742, -0.533233, -1.0},
{0.5, 2.004168, -1.0},
{0.5, 2.80528, -1.0},
},
{
{-0.5, -2.32594, -1.0},
{-0.5, -1.483963, -1.0},
{-0.499998, 0.008258, -1.0},
{0.4601

In [110]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for iphi in range(N_phi):
    if not is_sigmalised and not new: dphi.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
    if new and not is_sigmalised: dphi.append( round(0.002*phi_bins[2*iphi]-0.001-0.1,5))
    if is_sigmalised: dphi.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1])
[[array('d', [0.8, 0.8, 0.8, 0.8, 0.8, 0.8]), array('d', [-0.8, -0.8, -0.8, -0.8, -0.8, -0.8]), array('d', [-0.49999999958746405, -0.4999999989262648, -0.434834743009934, 0.42074184274937615, 0.4999998825510814, 0.4999999999956339]), array('d', [-0.4999999999763425, -0.4999999999208213, -0.4999980510593504, 0.4601648918891899, 0.49999990303135505, 0.4999999126180441]), array('d', [-0.08401362138383496, -0.018596589540063022, 0.21949096331149134, 0.374168959368289, -0.49061584349813825, 0.4999343712301098]), array('d', [-0.8, 0.0011940435637319653, -0.8, 0.11234269174815249, 0.03043704628597797, -0.06124156485879398]), array('d', [-0.499999896956627, -0.4999992294681155, -0.49999995602488667, -0.11021206233750007, 0.2381045681369753, 0.2687479819818765]), array('d', [-0.49999999931003947, -0.49999999409987744, -0.4999993268046149, -0.12001744200615458, 0.3320417091333317, 0.30466182703873296]), array('d', [0.8, 0.8, 0.8, 0.8, 0.8, 0.

In [111]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "mean [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, dynamic_dphi[ifile][ilayer],dphi_err,dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_mean_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_mean_phi[ifile][ilayer][iphi][-1].Fit("pol1","Q","")
            g_mean_phi[ifile][ilayer][iphi][-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
            g_mean_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_phi_{ifile}{iOption0}{is_sigmalised}.png")

In [112]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_functions[ifile][ilayer][iphi].Eval(pt_point)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
            dynamic_dphi_err_ilayer.append((fit_functions[ifile][ilayer][iphi].GetParError(0)*2)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [0.020804994160219668, 0.2529035701496845, 0.6468521245848965, 1.3531478754151034, 0.12370083765361953, 0.38515047477059255]), array('d', [0.05570331323022356, 0.028036417075549505, 1.8209442762135912, 0.17905572378640858, 0.12341163492570979, 0.39236168859039344]), array('d', [4.6764151776907035, 1.0234679795669537, 1.0019027458481686, 0.9980972541518313, 1.0206997669251936, 1.0061104510252263]), array('d', [1.0732964375152452, 1.03244141180396, 1.0080400088881403, 0.9919599911118601, 1.0151044856789915, 1.016440508422801]), array('d', [0.2752627064865822, 2.6764012993379467, 0.862922490703377, 1.137077509296623, 0.40291241139234535, 0.8320744313357699]), array('d', [0.05517899711122562, 0.07315286218584427, 1.6079048476370807, 0.39209515236291925, 0.3408210817797618, 0.23832847344869823]), array('d', [0.9957110690512359, 1.0130796090205798, 0.9956169230651821, 1.0043830769348179, 0.9519148100134796, 0.9585776895779302]), array('d', [1.0004305867647914, 0.991371329431103,

In [113]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "sigma [rad]", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, sigma_dynamic_dphi[ifile][ilayer],dphi_err,sigma_dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_sigma_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma_phi[ifile][ilayer][iphi][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][ilayer][iphi][-1].GetFunction("pol2").SetLineColor(colors[ilayer%4])
            g_sigma_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/sigma_phi_{ifile}{iOption0}{is_sigmalised}.png")

In [114]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_layers}][3]="+"\n{")
for fit_functions_ifile in fit_functions:
    print("    {")
    for fits in fit_functions_ifile:
        print("        {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("    },")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][3]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(0),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(1),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("    },")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][2]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")



const float phi_sigma_pt_params[1][16][3]=
{
    {
        {0.787312, 0.005648, -5.186421},
        {0.726061, 0.418363, -4.743499},
        {0.094843, 0.969926, -0.216485},
        {0.170082, 1.005782, -0.246399},
        {0.04966, 8.871163, -7.020541},
        {0.12241, 10.03311, -7.419172},
        {0.230079, 0.745719, 0.039467},
        {0.250942, 0.74435, 0.010492},
        {0.190667, 0.100506, -1.909533},
        {0.242435, 0.020337, 1.678894},
        {0.222261, 0.86724, -0.020524},
        {5.3e-05, 0.961851, 0.052923},
        {0.000326, 15.497957, -9.125444},
        {0.056548, 1.547471, -3.956073},
        {0.151577, 0.377837, 4.648284},
        {0.090715, 1.023799, -0.565903},
    },
};
const float phi_sigma_phi_params[1][16][3]=
{
    {
        {103727887859.88913, -195427646268.794, -548066124471.4872},
        {103727887859.88913, -195427646268.794, -548066124471.4872},
        {103727887859.88913, -195427646268.794, -548066124471.4872},
        {103727887859.88913, -195

In [115]:
outrootfile = root.TFile(f"output/dynamic_delta_fits/dynamicdeltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_layer in g_sigma_f:
        for g_sigma_phi in g_sigma_layer:
            g_sigma_phi.Write()